In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Set paths to your dataset
dataset_dir = "./dataset-split"
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

# Image dimensions
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:100]:  # Adjust the number of layers to freeze
    layer.trainable = False

# Compile again for fine-tuning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE / 10),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2861 - loss: 1.7424

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2849 - loss: 1.7404 - val_accuracy: 0.2555 - val_loss: 1.4669 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.2542 - loss: 1.6760 - val_accuracy: 0.3723 - val_loss: 1.3806 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3169 - loss: 1.4670 - val_accuracy: 0.3796 - val_loss: 1.3437 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.3553 - loss: 1.3936 - val_accuracy: 0.4307 - val_loss: 1.2887 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.4271 - loss: 1.2749 - val_accuracy: 0.4453 - val_loss: 1.2477 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.3938 - loss: 1.2902 - val_accuracy: 0.4526 - val_loss: 1.2253 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4415 - loss: 1.2255 - v

In [ ]:
import splitfolders  

# Path to the dataset containing category folders
original_dataset_dir = r"./dataset"
output_dir = r"./dataset-split"

# Split dataset into train (70%), val (20%), and test (10%)
splitfolders.ratio(
    original_dataset_dir, 
    output=output_dir, 
    seed=42, 
    ratio=(0.7, 0.2, 0.1), 
    group_prefix=None,  # Keeps images with similar prefix in the same set
    move=False  
)


In [5]:
# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)



Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.4686 - loss: 1.2591 - val_accuracy: 0.4964 - val_loss: 1.0800 - learning_rate: 1.0000e-05
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.4418 - loss: 1.2767 - val_accuracy: 0.5036 - val_loss: 1.1017 - learning_rate: 1.0000e-05
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5054 - loss: 1.1543 - val_accuracy: 0.5109 - val_loss: 1.1253 - learning_rate: 1.0000e-05
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5107 - loss: 1.0800 - val_accuracy: 0.5255 - val_loss: 1.1400 - learning_rate: 1.0000e-05
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5597 - loss: 1.0212 - val_accuracy: 0.5255 - val_loss: 1.1386 - learning_rate: 2.0000e-06


In [6]:
# Evaluate the model
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)



Found 74 images belonging to 4 classes.


In [7]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step - accuracy: 0.4989 - loss: 1.1386
Test Accuracy: 0.51


In [ ]:

# Save the model
model.save("children_drawings_model.h5")
